<a href="https://colab.research.google.com/github/smpark0520/ESAA/blob/main/3_7_%EC%84%B8%EC%85%98_%EB%B6%84%EB%A5%98_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **분류 연습 문제**
___
출처 : 핸즈온 머신러닝 Ch03 분류 연습문제 1, 2번

In [1]:
# import data
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1, as_frame = False)

In [2]:
X, y = mnist["data"], mnist["target"]

In [3]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

### **1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.**
___

1. `KNeghtborsClassifier`를 사용하는 것을 추천합니다.
2. `weights`와 `n_neighbors` 하이퍼 파라미터로 그리드 탐색을 시도하여, 좋은 하이퍼 파라미터 값을 찾아보세요.

In [19]:
# import package
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
# Try GridSearch to optimize hyperparameter
param_grid = {
    "n_neighbors": [3, 5, 7, 9],
    "weights": ["uniform", "distance"]
}

knn_clf = KNeighborsClassifier()

grid_search = GridSearchCV(knn_clf, param_grid, cv=3, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [3, 5, 7, 9],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [16]:
# best hyperparameter
best_knn = grid_search.best_estimator_
best_knn

KNeighborsClassifier(weights='distance')

In [17]:
# best score
grid_search.best_score_

0.9420999999999999

In [20]:
# model test
y_pred = best_knn.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test set accuracy: {accuracy:.4f}")

Test set accuracy: 0.9450


### **2. 다음 단계를 따라 인위적으로 훈련 세트를 늘리는 데이터 증식 또는 훈련 세트 확장 기법을 연습해봅시다.**
___

#### **STEP 1. MNIST 이미지를 (왼, 오른, 위, 아래) 어느 방향으로든 한 픽셀 이동시킬 수 있는 함수를 만들어 보세요.**

In [21]:
import numpy as np

In [22]:
def shift_image(image, direction):
    """
    이미지를 한 픽셀 이동시키는 함수입니다.
    방향에 따라 이미지를 이동시킵니다:
    'left', 'right', 'up', 'down'

    :param image: 28x28 크기의 numpy 배열
    :param direction: 이동할 방향 ('left', 'right', 'up', 'down')
    :return: 이동된 이미지
    """
    # 28x28 크기의 빈 이미지 (0으로 초기화)
    shifted_image = np.zeros_like(image)

    if direction == 'left':
        shifted_image[:, 1:] = image[:, :-1]  # 왼쪽으로 이동
    elif direction == 'right':
        shifted_image[:, :-1] = image[:, 1:]  # 오른쪽으로 이동
    elif direction == 'up':
        shifted_image[1:, :] = image[:-1, :]  # 위로 이동
    elif direction == 'down':
        shifted_image[:-1, :] = image[1:, :]  # 아래로 이동
    else:
        raise ValueError("Direction must be one of: 'left', 'right', 'up', 'down'")

    return shifted_image

####  **STEP 2. 앞에서 만든 함수를 이용하여, 훈련 세트에 있는 각 이미지에 대해 네 개의 이동된 복사본(방향마다 한 개씩)을 만들어 훈련 세트에 추가하세요**

In [25]:
# 이동된 이미지를 저장할 리스트
X_train_augmented = []

# 각 이미지를 왼쪽, 오른쪽, 위, 아래로 이동시켜서 추가
for image in X_train:  # X_train은 이미 NumPy 배열이므로 바로 사용
    # 이미지 복사본 생성
    image = image.reshape(28, 28)  # 1D 벡터를 28x28 행렬로 변환

    # 원본 이미지 + 4개의 이동된 이미지
    X_train_augmented.append(image)  # 원본 추가
    X_train_augmented.append(shift_image(image, 'left'))  # 왼쪽으로 이동
    X_train_augmented.append(shift_image(image, 'right'))  # 오른쪽으로 이동
    X_train_augmented.append(shift_image(image, 'up'))  # 위로 이동
    X_train_augmented.append(shift_image(image, 'down'))  # 아래로 이동

# numpy 배열로 변환
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.tile(y_train, 5)

In [26]:
# 결과 출력 (훈련 세트 크기 확인)
print(f"Original training set size: {X_train.shape[0]}")
print(f"Augmented training set size: {X_train_augmented.shape[0]}")

Original training set size: 60000
Augmented training set size: 300000


####  **STEP 3. 위에서 확장한 데이터셋을 이용하여, 1번 문제에서 찾은 최적 모델을 훈련시키고, 테스트 세트에서 정확도를 측정해보세요**

In [27]:
# 훈련 세트 확장된 데이터셋과 레이블 사용
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.tile(y_train, 5)

In [29]:
# 이미지를 1D 벡터로 변환 (각 이미지는 28x28, 이를 784차원 벡터로 변환)
X_train_augmented = X_train_augmented.reshape(X_train_augmented.shape[0], -1)  # (num_samples, 784)
X_test_scaled = X_test.reshape(X_test.shape[0], -1)

# 데이터 표준화 (스케일링)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_augmented)
X_test_scaled = scaler.transform(X_test_scaled)

In [31]:
# 최적의 하이퍼파라미터를 찾은 후
best_params = grid_search.best_params_  # GridSearchCV에서 최적의 파라미터 가져오기

# 최적의 하이퍼파라미터로 KNN 분류기 훈련
best_knn = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], weights=best_params['weights'])

# 모델 훈련
best_knn.fit(X_train_scaled, y_train_augmented)

# 모델 테스트
y_pred = best_knn.predict(X_test_scaled)

# 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"Test set accuracy with augmented data: {accuracy:.4f}")

Test set accuracy with augmented data: 0.1051
